# Imports
<a id='imports_id'> </a>

In [2]:
import os
import json

import pandas as pd
from dataclasses import dataclass, field
from typing import TextIO, List
from abc import ABC
import pyarrow as pa
from functools import cached_property


In [5]:
PREFIX = 'sent'
DATA_FOLDER = 'data'
RAW_DATA = 'SentenceDatabase.txt'

# POS Data Extraction

In [17]:
@dataclass
class DatabaseParser(ABC):
    dest: str
    seq_per_csv: int = field(default=100000)


    @cached_property
    def table_schema(self) -> pa.Schema:
        list_str_type = pa.list_(pa.string())
        table_schema = pa.schema([
            pa.field('init_words', list_str_type),
            pa.field('mod_words', list_str_type),
            pa.field('pos_tags', list_str_type)])
        return table_schema

    def __parse_word(self, line: str) -> List[str]:
        return line.split()[1:]  # discard word index

    def __read_sentence(self, database: TextIO) -> pa.Table:
        init_words = []
        mod_words = []
        pos_tags = []

        for i, line in enumerate(database):
            if line == "":
                raise EOFError
            if not line[0].isnumeric():
                break
            init_word, mod_word, pos_tag = self.__parse_word(line)
            init_words.append(init_word)
            mod_words.append(mod_word)
            pos_tags.append(pos_tag)

        return pa.Table.from_arrays([[init_words], [mod_words], [pos_tags]], schema=self.table_schema)

    def parse(self, database: TextIO) -> None:
        sentences = []
        curr: int = 0
        part: int = 1
        while True:
            try:
                curr_sent = self.__read_sentence(database)
            except EOFError:
                break
            curr += 1
            if curr == self.seq_per_csv:
                pa.concat_tables(sentences).to_pandas(types_mapper=pd.ArrowDtype).to_orc(os.path.join(self.dest, f'{PREFIX}_{part}.orc'))
                sentences = [curr_sent]
                curr = 0
                part += 1
            else:
                sentences.append(curr_sent)


        sentences.to_pandas(types_mapper=pd.ArrowDtype).to_orc(os.path.join(self.dest, f'{PREFIX}_{part}.orc'))




In [ ]:
with open(os.path.join(DATA_FOLDER, RAW_DATA), 'r') as f:
   DatabaseParser(DATA_FOLDER, 1000000).parse(f)

In [6]:
NUM_PARTS = 0

for root, dirs, files in  os.walk(DATA_FOLDER):
    NUM_PARTS += len([file for file in files if file.startswith(PREFIX)])

NUM_PARTS

9

In [27]:
def get_dataset(idx: int, prefix: str) -> pd.DataFrame:
    return pd.read_orc(os.path.join(DATA_FOLDER, f'{prefix}_{idx}.orc'))

# Frequency Data

In [43]:
from collections import Counter
def get_freq(prefix: str):
    freq = Counter()
    for idx in range(1, NUM_PARTS + 1):
        df = get_dataset(idx, prefix)
        for word in df['mod_words'].explode():
            freq[word] += 1
    return freq

In [ ]:
freq = get_freq(PREFIX)

In [10]:
with open(os.path.join(DATA_FOLDER, 'vocab', 'freq.json'), 'w') as f:
    json.dump(freq, f, ensure_ascii = False)

In [11]:
# Freq. Score

In [71]:
freq_scores = []
for idx in range(1, NUM_PARTS + 1):
    df = get_dataset(idx, PREFIX)
    min_score = df.mod_words.apply(lambda l: min([freq[word] for word in l]) if len(l) > 0 else 0).quantile(0.02)
    max_score = df.mod_words.apply(lambda l: max([freq[word] for word in l]) if len(l) > 0 else 0).quantile(0.98)
    avg_score = df.mod_words.apply(lambda l: sum([freq[word] for word in l]) / len(l) if len(l) > 0 else 0).quantile(0.02)
    freq_scores.append((min_score, avg_score, max_score))

In [73]:
max_freq_score = min([score[2] for score in freq_scores])
max_freq_score

11924561.0

In [76]:
min_avg_freq_score = sum([score[1] for score in freq_scores]) / len(freq_scores)
min_avg_freq_score

382749.2299380481

In [74]:
min_freq_score = max([score[0] for score in freq_scores])
min_freq_score

1.0

## Clear Data

In [59]:
CLEARED_PREFIX = f'cleared_{PREFIX}'

In [64]:
rest = []
for idx in range(1, NUM_PARTS + 1):
    df = get_dataset(idx, PREFIX)
    df_mod = df[df.mod_words.apply(lambda l: min([freq[word] for word in l]) if len(l) > 0 else 0) > min_freq_score]
    df_mod = df_mod[~df_mod.pos_tags.apply(lambda l: 'Other' in set(l))]
    df_mod.to_orc(os.path.join(DATA_FOLDER, f'{CLEARED_PREFIX}_{idx}.orc'))
    rest_df = df[df.mod_words.apply(lambda l: min([freq[word] for word in l]) if len(l) > 0 else 0) <= min_freq_score]
    rest.append(rest_df)


In [69]:
for idx, sent in enumerate(pd.concat(rest).mod_words):
    if len(sent) < 10:
        print(" ".join(sent))
    if idx >= 200:
        break

მისი ზურგი ათეულობით ადამიანის მწყობრისკენაა მიქცეული .
ფეხები ლურსმნებივით დამჭედებოდა .
მას უნდა ეზღო ხაიმკესათვის .
სტაშეკმაც დაიჭირა მილკას გახარებული მზერა და გაეღიმა .
ფელდმანი ჩენსტუხოვიდან , სტაშეკ !
ფრენისა და თავბრუხვევისაგან დაქანცულიყო .
როხელეს საქმრო თავის ხელისმომკიდესთან ერთად გაღიმებული შემოდის .
ჩვენი თვალები ელვარებისგან თითქოს ნაპერწკლებს ყრიან .
სარკიდანვე , ჩემი შეძრწუნებული სახე მომჩერებია .
როგორ ფიქრობთ ამდენი ხალხი ზღვებს ცუდი ექიმისათვის გადმოცურავდა ?
შარვლის ტოტები დაეკარწახებინა , წელზემოთ შიშველი იყო .
ფანდურზედაც დაამღერა და უფრო ძაან მოეწონა .
… ეულალიას ქმარი !
მე შენე არა ვარ .
შლაპაზე ბუზი დამჯდარიყო , ხელი ავუქნიე .
თქვენ საითკენღა წახვედით , შალვა ?
შრიალ-შრიალში , ოღროჩოღროზე მივაბიჯებდი , გული მიცემდა .
… ვხვანჩალობდით , ჰეჰ !
რაღაც მაპეტიჟებოდი აქეთ … მე შენ გიჩვენებ !
და ისევ ის არ მომიგო , მრავალპირიანი ?
წარმოიდგინეთ , მიკრო-უდაბნოც კი მოიძევება .
თვალის ერთი გადავლებით შემეძლო გამერჩია ჩინეთი არიზონისაგან .
მაგალითად , „ ასტეროიდი 3251 ” .
ნ

In [66]:
get_freq(CLEARED_PREFIX)

Counter({',': 10245863,
         '.': 7313044,
         'და': 3254784,
         'რომ': 1516287,
         '„': 1395447,
         'არ': 1159666,
         '“': 841077,
         'ეს': 692591,
         'ამ': 691970,
         'კი': 491561,
         'უნდა': 472451,
         '”': 456784,
         'თუ': 440344,
         'მაგრამ': 414166,
         'იყო': 407043,
         'საქართველოს': 401173,
         'არის': 371985,
         ':': 344858,
         'ის': 330532,
         'როგორც': 315742,
         'რომელიც': 292213,
         'მისი': 275717,
         'შემდეგ': 262382,
         'რაც': 261827,
         ')': 244192,
         '?': 241701,
         'წლის': 236443,
         '(': 234999,
         'რა': 233066,
         'ვერ': 221531,
         'არა': 221036,
         'იმ': 215448,
         'თუმცა': 209196,
         'მათ': 206744,
         '!': 205224,
         'ერთი': 203486,
         'მას': 193764,
         'მხოლოდ': 192024,
         'არც': 178359,
         '…': 176778,
         'მე': 176676,
         '

# Vocabulary Exctraction

In [47]:
def get_vocab(prefix: str):
    unique_words = set()
    for idx in range(1, NUM_PARTS + 1):
        df = get_dataset(idx, prefix)
        unique_words |= set(df['mod_words'].explode())
    return unique_words

In [11]:
unique_words = get_vocab(PREFIX)
len(unique_words)

1836994

In [18]:
with open(os.path.join(DATA_FOLDER, 'vocab', 'words.json'), 'w') as f:
    json.dump(list(unique_words), f, ensure_ascii = False)

In [67]:
unique_words = get_vocab(CLEARED_PREFIX)
len(unique_words)

904401